# Alignment and variant calling

## Getting the files

In [1]:
cp -r  /scratch/work/courses/AppliedGenomics2021Sec3/week03_SeqAln /scratch/ycc520/ag_recitation/wk5_recitation/data
ls -la data

total 80385
drwxr-sr-x 2 ycc520 ycc520     4096 Mar  1 07:51 .
drwxr-sr-x 4 ycc520 ycc520     4096 Mar  1 07:50 ..
-rw-r-xr-- 1 ycc520 ycc520 65518251 Mar  1 07:50 GCF_000001405.33_GRCh38.p7_chr20_genomic.fna
-rw-r-xr-- 1 ycc520 ycc520 23476243 Mar  1 07:51 read_1.fastq
-rw-r-xr-- 1 ycc520 ycc520 23476243 Mar  1 07:51 read_2.fastq


## Loading modules

In [1]:
# There modules that are loaded in Jupyter by default
# But not in the terminal

module list


Currently Loaded Modules:
  1) python/intel/3.8.6   2) intel/19.1.2   3) r/intel/4.0.3

 



In [1]:
module purge

module load bwa/intel/0.7.17
module load picard/2.23.8
module load samtools/intel/1.11
module load gatk/4.1.9.0
module load snpeff/4.3t
module load trimmomatic/0.39

module list


Currently Loaded Modules:
  1) bwa/intel/0.7.17      5) samtools/intel/1.11   9) gatk/4.1.9.0
  2) picard/2.23.8         6) python/intel/3.8.6   10) snpeff/4.3t
  3) perl/intel/5.32.0     7) intel/19.1.2         11) jdk/1.8.0_271
  4) htslib/intel/1.11.0   8) r/intel/4.0.3        12) trimmomatic/0.39

 



## Indexing the genome

In [4]:
cd data

In [5]:
bwa index GCF_000001405.33_GRCh38.p7_chr20_genomic.fna

[bwa_index] Pack FASTA... 0.43 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=128888334, availableWord=21068624
[BWTIncConstructFromPacked] 10 iterations done. 34753182 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 64202446 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 90372990 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 113629422 characters processed.
[bwt_gen] Finished constructing BWT in 48 iterations.
[bwa_index] 32.75 seconds elapse.
[bwa_index] Update BWT... 0.30 sec
[bwa_index] Pack forward-only FASTA... 0.26 sec
[bwa_index] Construct SA from BWT and Occ... 15.42 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index GCF_000001405.33_GRCh38.p7_chr20_genomic.fna
[main] Real time: 49.839 sec; CPU: 49.166 sec


## To trim or not to trim, that's the question.

In [12]:
cd data
java -jar $TRIMMOMATIC_JAR PE -phred33 \
read_1.fastq read_2.fastq \
read_1_trimmed.fq read_1_unpair_trimmed.fq \
read_2_trimmed.fq read_2_unpair_trimmed.fq \
HEADCROP:15 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
cd ..

TrimmomaticPE: Started with arguments:
 -phred33 read_1.fastq read_2.fastq read_1_trimmed.fq read_1_unpair_trimmed.fq read_2_trimmed.fq read_2_unpair_trimmed.fq HEADCROP:15 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
Multiple cores found: Using 4 threads
Input Read Pairs: 97206 Both Surviving: 88187 (90.72%) Forward Only Surviving: 5936 (6.11%) Reverse Only Surviving: 1671 (1.72%) Dropped: 1412 (1.45%)
TrimmomaticPE: Completed successfully


In [13]:
ls -l data

total 257409
-rw-r-xr-- 1 ycc520 ycc520 65518251 Mar  1 07:50 GCF_000001405.33_GRCh38.p7_chr20_genomic.fna
-rw-rw-r-- 1 ycc520 ycc520     1341 Mar  1 07:55 GCF_000001405.33_GRCh38.p7_chr20_genomic.fna.amb
-rw-rw-r-- 1 ycc520 ycc520       50 Mar  1 07:55 GCF_000001405.33_GRCh38.p7_chr20_genomic.fna.ann
-rw-rw-r-- 1 ycc520 ycc520 64444268 Mar  1 07:55 GCF_000001405.33_GRCh38.p7_chr20_genomic.fna.bwt
-rw-rw-r-- 1 ycc520 ycc520 16111043 Mar  1 07:55 GCF_000001405.33_GRCh38.p7_chr20_genomic.fna.pac
-rw-rw-r-- 1 ycc520 ycc520 32222136 Mar  1 07:56 GCF_000001405.33_GRCh38.p7_chr20_genomic.fna.sa
-rw-r-xr-- 1 ycc520 ycc520 23476243 Mar  1 07:51 read_1.fastq
-rw-rw-r-- 1 ycc520 ycc520 18246659 Mar  1 08:00 read_1_trimmed.fq
-rw-rw-r-- 1 ycc520 ycc520  1105956 Mar  1 08:00 read_1_unpair_trimmed.fq
-rw-r-xr-- 1 ycc520 ycc520 23476243 Mar  1 07:51 read_2.fastq
-rw-rw-r-- 1 ycc520 ycc520 18051041 Mar  1 08:00 read_2_trimmed.fq
-rw-rw-r-- 1 ycc520 ycc520   305070 Mar  1 08:00 read_2_unpair_trimmed.f

In [15]:
mkdir result

In [16]:
## Align using the reference genome
bwa mem data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna \
data/read_1_trimmed.fq data/read_2_trimmed.fq > result/aligned_reads.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 121938 sequences (10000101 bp)...
[M::process] read 54436 sequences (4487916 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (5, 45565, 0, 3)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (293, 313, 333)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (213, 413)
[M::mem_pestat] mean and std.dev: (313.06, 26.68)
[M::mem_pestat] low and high boundaries for proper pairs: (173, 453)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 121938 reads in 9.419 CPU sec, 9.436 real sec
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (1, 20392, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] a

In [18]:
## Sorting and converting with PICARD
java -jar $PICARD_JAR SortSam \
          INPUT=result/aligned_reads.sam \
          OUTPUT=result/sorted_reads.bam \
          SORT_ORDER=coordinate

INFO	2021-03-01 08:02:36	SortSam	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    SortSam -INPUT result/aligned_reads.sam -OUTPUT result/sorted_reads.bam -SORT_ORDER coordinate
**********


08:02:36.329 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Mon Mar 01 08:02:36 EST 2021] SortSam INPUT=result/aligned_reads.sam OUTPUT=result/sorted_reads.bam SORT_ORDER=coordinate    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json USE_JDK_DEFLATER=false USE_JDK_INFLATER=false
[Mon M

In [19]:
samtools flagstat result/sorted_reads.bam

176401 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
27 + 0 supplementary
0 + 0 duplicates
176272 + 0 mapped (99.93% : N/A)
176374 + 0 paired in sequencing
88187 + 0 read1
88187 + 0 read2
174110 + 0 properly paired (98.72% : N/A)
176122 + 0 with itself and mate mapped
123 + 0 singletons (0.07% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [22]:
## Add read group to the bam file
java -jar $PICARD_JAR AddOrReplaceReadGroups \
          I=result/sorted_reads.bam \
          O=result/sorted_reads_rg.bam \
          RGID=sample_1 \
          RGLB=sample_1 \
          RGPL=ILLUMINA \
          RGPM=HISEQ \
          RGPU=sample_1 \
          RGSM=sample_1

INFO	2021-03-01 08:12:38	AddOrReplaceReadGroups	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    AddOrReplaceReadGroups -I result/sorted_reads.bam -O result/sorted_reads_rg.bam -RGID sample_1 -RGLB sample_1 -RGPL ILLUMINA -RGPM HISEQ -RGPU sample_1 -RGSM sample_1
**********


08:12:39.019 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Mon Mar 01 08:12:39 EST 2021] AddOrReplaceReadGroups INPUT=result/sorted_reads.bam OUTPUT=result/sorted_reads_rg.bam RGID=sample_1 RGLB=sample_1 RGPL=ILLUMINA RGPU=sample_1 RGSM=sample_1 RGPM=HISEQ    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL

### Information in the header

In [25]:
samtools view -H result/sorted_reads.bam

@HD	VN:1.6	SO:coordinate
@SQ	SN:NC_000020.11	LN:64444167
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna data/read_1_trimmed.fq data/read_2_trimmed.fq
@PG	ID:samtools	PN:samtools	PP:bwa	VN:1.11	CL:samtools view -H result/sorted_reads.bam


In [26]:
samtools view -H result/sorted_reads_rg.bam

@HD	VN:1.6	SO:coordinate
@SQ	SN:NC_000020.11	LN:64444167
@RG	ID:sample_1	LB:sample_1	PL:ILLUMINA	SM:sample_1	PU:sample_1	PM:HISEQ
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna data/read_1_trimmed.fq data/read_2_trimmed.fq
@PG	ID:samtools	PN:samtools	PP:bwa	VN:1.11	CL:samtools view -H result/sorted_reads_rg.bam


### Information per read

In [23]:
samtools view result/sorted_reads.bam | grep -v "##" | head -n 1

HS2000-940_146:5:2109:14063:29918	161	NC_000020.11	64145	4	39S46M	=	23724992	23660929	TTCCATCACACTGCATTCCATTCCATTCCAATCCCCTCAACTCCACTCCACTCCACTCCATTCCACTCCAATCAATTCCATTGCA	JJJJJIJIFHHGCGIHHIJJJGHIHIIIJJIHIGGGHIJJE:FFHIGIDHJGIGGIJJ@;CDHGGEIHHHEHF;CCB>;;3;>;>	XA:Z:NC_000020.11,+60520,39S31M15S,0;NC_000020.11,+60582,41S29M15S,0;NC_000020.11,+64492,41S29M15S,0;	MC:Z:82M	MD:Z:6G27C8T2	NM:i:3	AS:i:33	XS:i:31
grep: write error: Broken pipe
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1


In [24]:
samtools view result/sorted_reads_rg.bam | grep -v "##" | head -n 1

HS2000-940_146:5:2109:14063:29918	161	NC_000020.11	64145	4	39S46M	=	23724992	23660929	TTCCATCACACTGCATTCCATTCCATTCCAATCCCCTCAACTCCACTCCACTCCACTCCATTCCACTCCAATCAATTCCATTGCA	JJJJJIJIFHHGCGIHHIJJJGHIHIIIJJIHIGGGHIJJE:FFHIGIDHJGIGGIJJ@;CDHGGEIHHHEHF;CCB>;;3;>;>	XA:Z:NC_000020.11,+60520,39S31M15S,0;NC_000020.11,+60582,41S29M15S,0;NC_000020.11,+64492,41S29M15S,0;	MC:Z:82M	MD:Z:6G27C8T2	RG:Z:sample_1	NM:i:3	AS:i:33	XS:i:31
grep: write error: Broken pipe
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1


### What are read groups?

Source: [Gencore](https://learn.gencore.bio.nyu.edu/ngs-file-formats/sambam-format/)


![](https://learn.gencore.bio.nyu.edu/wp-content/uploads/2018/01/Screen-Shot-2018-01-07-at-4.47.54-PM-768x762.png)

![](https://learn.gencore.bio.nyu.edu/wp-content/uploads/2018/01/Screen-Shot-2018-01-07-at-4.51.29-PM-768x520.png)

## Mark duplicated reads

In [27]:
java -jar $PICARD_JAR MarkDuplicates \
          INPUT=result/sorted_reads_rg.bam \
          OUTPUT=result/dedup_reads.bam \
          METRICS_FILE=metrics.txt

INFO	2021-03-01 08:15:32	MarkDuplicates	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    MarkDuplicates -INPUT result/sorted_reads_rg.bam -OUTPUT result/dedup_reads.bam -METRICS_FILE metrics.txt
**********


08:15:33.008 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Mon Mar 01 08:15:33 EST 2021] MarkDuplicates INPUT=[result/sorted_reads_rg.bam] OUTPUT=result/dedup_reads.bam METRICS_FILE=metrics.txt    MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP=50000 MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=8000 SORTING_COLLECTION_SIZE_RATIO=0.25 TAG_DUPLICATE_SET_MEMBERS=false REMOVE_SEQUENCING_DUPLICATES=false TAGGING_POLICY=DontT

In [29]:
samtools view result/sorted_reads_rg.bam | grep HS2000-1010_101:8:2205:14144:55120

HS2000-1010_101:8:2205:14144:55120	161	NC_000020.11	24013196	60	85M	=	24013243	132	GTTCATTATTCCCCATGTTTCCTAAGATATGTTTTCTAAGCCAACACATTAGTTCAAATTACTGCATTTTTCTTGAATCTTGACA	DHEDHHGEJIIEGGGIGEEGHIIIIIJFGIEICFHFGIJGDDFADHEHBFHI;@F@GGFE@CDH@??ACA>@BDFD>?;A>6>C3	MC:Z:85M	MD:Z:25T59	RG:Z:sample_1	NM:i:1	AS:i:80	XS:i:0
HS2000-1010_101:8:2205:14144:55120	81	NC_000020.11	24013243	60	85M	=	24013196	-132	ATTAGTTCAAATTACTGCATTTTTCTTGAATCTTGACAAGAAAATTATGTAGGAAGTAGATTTGAGTTTTTGCGTAGCTGTGTCT	CCCCCCC@>DC@CACECCCBB@>@ACHHEHDD@IHEIIIHIF>GEIIIHGHIGGIIGFBFACGHGGHEBGGHGGHFEDFCDIIIG	MC:Z:85M	MD:Z:73A11	RG:Z:sample_1	NM:i:1	AS:i:80	XS:i:19


In [30]:
samtools view result/dedup_reads.bam | grep HS2000-1010_101:8:2205:14144:55120

HS2000-1010_101:8:2205:14144:55120	1185	NC_000020.11	24013196	60	85M	=	24013243	132	GTTCATTATTCCCCATGTTTCCTAAGATATGTTTTCTAAGCCAACACATTAGTTCAAATTACTGCATTTTTCTTGAATCTTGACA	DHEDHHGEJIIEGGGIGEEGHIIIIIJFGIEICFHFGIJGDDFADHEHBFHI;@F@GGFE@CDH@??ACA>@BDFD>?;A>6>C3	MC:Z:85M	MD:Z:25T59	PG:Z:MarkDuplicates	RG:Z:sample_1	NM:i:1	AS:i:80	XS:i:0
HS2000-1010_101:8:2205:14144:55120	1105	NC_000020.11	24013243	60	85M	=	24013196	-132	ATTAGTTCAAATTACTGCATTTTTCTTGAATCTTGACAAGAAAATTATGTAGGAAGTAGATTTGAGTTTTTGCGTAGCTGTGTCT	CCCCCCC@>DC@CACECCCBB@>@ACHHEHDD@IHEIIIHIF>GEIIIHGHIGGIIGFBFACGHGGHEBGGHGGHFEDFCDIIIG	MC:Z:85M	MD:Z:73A11	PG:Z:MarkDuplicates	RG:Z:sample_1	NM:i:1	AS:i:80	XS:i:19


### Explain flags

https://broadinstitute.github.io/picard/explain-flags.html

## Prepare reference dictionary, fasta index, and bam index

In [32]:
java -jar $PICARD_JAR CreateSequenceDictionary \
          R=data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna \
          O=data/GCF_000001405.33_GRCh38.p7_chr20_genomic.dict

INFO	2021-03-01 08:17:57	CreateSequenceDictionary	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    CreateSequenceDictionary -R data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna -O data/GCF_000001405.33_GRCh38.p7_chr20_genomic.dict
**********


08:17:58.111 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/picard/2.23.8/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Mon Mar 01 08:17:58 EST 2021] CreateSequenceDictionary OUTPUT=data/GCF_000001405.33_GRCh38.p7_chr20_genomic.dict REFERENCE=data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna    TRUNCATE_NAMES_AT_WHITESPACE=true NUM_SEQUENCES=2147483647 VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVE

In [36]:
cat data/GCF_000001405.33_GRCh38.p7_chr20_genomic.dict

@HD	VN:1.6
@SQ	SN:NC_000020.11	LN:64444167	M5:b18e6c531b0bd70e949a7fc20859cb01	UR:file:/scratch/ycc520/ag_recitation/wk5_recitation/data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna


In [39]:
head data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna

>NC_000020.11
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


In [40]:
samtools view -H result/dedup_reads.bam

@HD	VN:1.6	SO:coordinate
@SQ	SN:NC_000020.11	LN:64444167
@RG	ID:sample_1	LB:sample_1	PL:ILLUMINA	SM:sample_1	PU:sample_1	PM:HISEQ
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna data/read_1_trimmed.fq data/read_2_trimmed.fq
@PG	ID:MarkDuplicates	VN:2.23.8	CL:MarkDuplicates INPUT=[result/sorted_reads_rg.bam] OUTPUT=result/dedup_reads.bam METRICS_FILE=metrics.txt    MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP=50000 MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=8000 SORTING_COLLECTION_SIZE_RATIO=0.25 TAG_DUPLICATE_SET_MEMBERS=false REMOVE_SEQUENCING_DUPLICATES=false TAGGING_POLICY=DontTag CLEAR_DT=true DUPLEX_UMI=false ADD_PG_TAG_TO_READS=true REMOVE_DUPLICATES=false ASSUME_SORTED=false DUPLICATE_SCORING_STRATEGY=SUM_OF_BASE_QUALITIES PROGRAM_RECORD_ID=MarkDuplicates PROGRAM_GROUP_NAME=MarkDuplicates READ_NAME_REGEX=<optimized capture of last three ':' separated fields as numeric values> OPTICAL_DUPLICATE_PIXEL_DISTANCE=100 MAX_OPTICAL_DUPLICATE_SET_SIZE=30000

In [41]:
samtools faidx data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna

In [43]:
samtools index result/dedup_reads.bam

## Forget something?

## Variant calling

In [44]:
java -jar $GATK_JAR HaplotypeCaller \
          -R data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna \
          -I result/dedup_reads.bam \
          -O result/raw_variants.vcf

08:23:44.212 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Mar 01, 2021 8:23:44 AM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
08:23:44.358 INFO  HaplotypeCaller - ------------------------------------------------------------
08:23:44.358 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.1.9.0
08:23:44.358 INFO  HaplotypeCaller - For support and documentation go to https://software.broadinstitute.org/gatk/
08:23:44.358 INFO  HaplotypeCaller - Executing as ycc520@cs045.nyu.cluster on Linux v4.18.0-193.28.1.el8_2.x86_64 amd64
08:23:44.358 INFO  HaplotypeCaller - Java runtime: Java HotSpot(TM) 64-Bit Server VM v1.8.0_271-b09
08:23:44.358 INFO  HaplotypeCaller - Start Date/Time: March 1, 2021 8:23:44 AM EST
08:23:44.358 INFO  HaplotypeCaller

### VCF format recap


![](https://learn.gencore.bio.nyu.edu/wp-content/uploads/2018/01/Screen-Shot-2018-01-07-at-9.29.53-PM-768x554.png)


> Ref: [Gencore](https://learn.gencore.bio.nyu.edu/ngs-file-formats/vcf-format/)

In [48]:
# split the variants to snps and indels
gatk SelectVariants \
     -R data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna \
     -V result/raw_variants.vcf \
     -select-type SNP \
     -O result/raw_snps.vcf
     
gatk SelectVariants \
     -R data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna \
     -V result/raw_variants.vcf \
     -select-type INDEL \
     -O result/raw_indels.vcf

Using GATK jar /share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar defined in environment variable GATK_LOCAL_JAR
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar SelectVariants -R data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna -V result/raw_variants.vcf -select-type SNP -O result/raw_snps.vcf
08:27:11.049 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Mar 01, 2021 8:27:11 AM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
08:27:11.160 INFO  SelectVariants - ------------------------------------------------------------
08:27:11.160 INFO  SelectVariants - The Genome 

In [49]:
java -jar $GATK_JAR VariantFiltration \
          -R data/GCF_000001405.33_GRCh38.p7_chr20_genomic.fna \
          -V result/raw_snps.vcf \
          --filter-name "QD_filter" \
          -filter "QD<2.0" \
          --filter-name "FS_filter" \
          -filter "FS>60.0" \
          --filter-name "MQ_filter" \
          -filter "MQ<40.0" \
          --filter-name "SOR_filter" \
          -filter "SOR>10.0" \
          -O result/filtered_snps.vcf

08:28:21.520 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/share/apps/gatk/4.1.9.0/gatk-package-4.1.9.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Mar 01, 2021 8:28:21 AM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
08:28:21.634 INFO  VariantFiltration - ------------------------------------------------------------
08:28:21.634 INFO  VariantFiltration - The Genome Analysis Toolkit (GATK) v4.1.9.0
08:28:21.634 INFO  VariantFiltration - For support and documentation go to https://software.broadinstitute.org/gatk/
08:28:21.634 INFO  VariantFiltration - Executing as ycc520@cs045.nyu.cluster on Linux v4.18.0-193.28.1.el8_2.x86_64 amd64
08:28:21.634 INFO  VariantFiltration - Java runtime: Java HotSpot(TM) 64-Bit Server VM v1.8.0_271-b09
08:28:21.634 INFO  VariantFiltration - Start Date/Time: March 1, 2021 8:28:21 AM EST
08:28:21.634 INFO  Var

### What exactly the filtering is looking at?

https://gatk.broadinstitute.org/hc/en-us/articles/360035890471-Hard-filtering-germline-short-variants

## Annotation

In [50]:
mkdir annotation

In [51]:
cp result/filtered_snps.vcf annotation

In [5]:
sed -i "s/NC_000020.11/20/g" annotation/filtered_snps.vcf

In [9]:
cat result/filtered_snps.vcf | grep -v "##" | head -n 2

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	sample_1
NC_000020.11	23531045	.	T	A	987.06	PASS	AC=2;AF=1.00;AN=2;DP=29;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=25.36;SOR=1.022	GT:AD:DP:GQ:PL	1/1:0,26:26:78:1001,78,0
grep: write error: Broken pipe
cat: write error: Broken pipe


In [10]:
cat annotation/filtered_snps.vcf | grep -v "##" | head -n 2

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	sample_1
20	23531045	.	T	A	987.06	PASS	AC=2;AF=1.00;AN=2;DP=29;ExcessHet=3.0103;FS=0.000;MLEAC=2;MLEAF=1.00;MQ=60.00;QD=25.36;SOR=1.022	GT:AD:DP:GQ:PL	1/1:0,26:26:78:1001,78,0
grep: write error: Broken pipe
cat: write error: Broken pipe


In [6]:
java -jar $SNPEFF_JAR \
     -v GRCh38.p7.RefSeq \
     annotation/filtered_snps.vcf > annotation/filtered_snps.ann.vcf

00:00:00	SnpEff version SnpEff 4.3t (build 2017-11-24 10:18), by Pablo Cingolani
00:00:00	Command: 'ann'
00:00:00	Reading configuration file 'snpEff.config'. Genome: 'GRCh38.p7.RefSeq'
00:00:00	Reading config file: /scratch/ycc520/ag_recitation/wk5_recitation/snpEff.config
00:00:00	Reading config file: /share/apps/snpeff/4.3t/snpEff.config
00:00:00	done
00:00:00	Reading database for genome version 'GRCh38.p7.RefSeq' from file '/scratch/work/cgsb/snpeff_data/GRCh38.p7.RefSeq/snpEffectPredictor.bin' (this might take a while)
00:00:35	done
00:00:35	Reading NextProt database from file '/scratch/work/cgsb/snpeff_data/GRCh38.p7.RefSeq/nextProt.bin'
00:00:36	NextProt database: 537115 markers loaded.
00:00:36	Adding transcript info to NextProt markers.
00:00:36	NextProt database: 537115 markers added.
00:00:36	Loading Motifs and PWMs
00:00:36	Loading interactions from : /scratch/work/cgsb/snpeff_data/GRCh38.p7.RefSeq/interactions.bin
00:00:42		Interactions: 1596432 added, 0 skipped.
00:00:42	B

In [57]:
head annotation/filtered_snps.ann.vcf